In [1]:
import gzip

In [2]:
f = gzip.open('data/ssh_log.gz', 'r')

In [3]:
t = f.readlines()
f.close()

In [4]:
t[0]

b'\xd1\x84\xd0\xb5\xd0\xb2 10 21:12:09 systemd[1]: Starting OpenBSD Secure Shell server...\n'

In [5]:
s = t[0]

In [6]:
s.decode('utf-8')

'фев 10 21:12:09 systemd[1]: Starting OpenBSD Secure Shell server...\n'

In [7]:
s = []
for i in t:
    s.append(i.decode('utf-8').strip())

In [8]:
s[:10]

['фев 10 21:12:09 systemd[1]: Starting OpenBSD Secure Shell server...',
 'фев 10 21:12:09 sshd[884]: Server listening on 0.0.0.0 port 22.',
 'фев 10 21:12:09 sshd[884]: Server listening on :: port 22.',
 'фев 10 21:12:09 systemd[1]: Started OpenBSD Secure Shell server.',
 'фев 10 21:12:21 sshd[1160]: pam_unix(sshd:auth): authentication failure; logname= uid=0 euid=0 tty=ssh ruser= rhost=221.131.165.86  user=root',
 'фев 10 21:12:24 sshd[1160]: Failed password for root from 221.131.165.86 port 49460 ssh2',
 'фев 10 21:12:28 sshd[1160]: Failed password for root from 221.131.165.86 port 49460 ssh2',
 'фев 10 21:12:32 sshd[1160]: Failed password for root from 221.131.165.86 port 49460 ssh2',
 'фев 10 21:12:33 sshd[1160]: Received disconnect from 221.131.165.86 port 49460:11:  [preauth]',
 'фев 10 21:12:33 sshd[1160]: Disconnected from authenticating user root 221.131.165.86 port 49460 [preauth]']

In [9]:
for i in s[:20]:
    print(i.split())

['фев', '10', '21:12:09', 'systemd[1]:', 'Starting', 'OpenBSD', 'Secure', 'Shell', 'server...']
['фев', '10', '21:12:09', 'sshd[884]:', 'Server', 'listening', 'on', '0.0.0.0', 'port', '22.']
['фев', '10', '21:12:09', 'sshd[884]:', 'Server', 'listening', 'on', '::', 'port', '22.']
['фев', '10', '21:12:09', 'systemd[1]:', 'Started', 'OpenBSD', 'Secure', 'Shell', 'server.']
['фев', '10', '21:12:21', 'sshd[1160]:', 'pam_unix(sshd:auth):', 'authentication', 'failure;', 'logname=', 'uid=0', 'euid=0', 'tty=ssh', 'ruser=', 'rhost=221.131.165.86', 'user=root']
['фев', '10', '21:12:24', 'sshd[1160]:', 'Failed', 'password', 'for', 'root', 'from', '221.131.165.86', 'port', '49460', 'ssh2']
['фев', '10', '21:12:28', 'sshd[1160]:', 'Failed', 'password', 'for', 'root', 'from', '221.131.165.86', 'port', '49460', 'ssh2']
['фев', '10', '21:12:32', 'sshd[1160]:', 'Failed', 'password', 'for', 'root', 'from', '221.131.165.86', 'port', '49460', 'ssh2']
['фев', '10', '21:12:33', 'sshd[1160]:', 'Received', 'd

In [10]:
fields = ['month', 'day', 'time', 'daemon', 'comment']

In [11]:
log = []
for i in s[:10]:
    sp = i.split()
    d = {}
    d['month']   = sp[0]
    d['day']     = sp[1]
    d['time']    = sp[2]
    d['daemon']  = sp[3]
    d['comment'] = ' '.join(sp[4:])
    log.append(d)
    

In [12]:
log = []
for n,i in enumerate(s):
    #print(n, end = ' ')
    sp = i.split()
    if len(sp) >= 5:
        d = {}
        d['month']   = sp[0]
        d['day']     = sp[1]
        d['time']    = sp[2]
        d['daemon']  = sp[3]
        d['comment'] = ' '.join(sp[4:])
        log.append(d)
    

In [13]:
psw = []
for z in log:
    if 'Failed password' in z['comment']:
        psw.append(z)

In [14]:
len(psw)

85372

In [15]:
psw_f = []
for p in psw:
    sp = p['comment'].split()
    if 'root' in sp:
        #print(sp)        
        p['user'] = sp[3]
        p['ip']   = sp[5]
        p['port'] = sp[7]
        p['attack'] = 'root'
        psw_f.append(p)
    elif 'invalid' in sp and 'user' in sp:
        #print(sp)
        p['user'] = sp[5]
        p['ip']   = sp[7]
        p['port'] = sp[9]
        p['attack'] = 'invalid user'
        psw_f.append(p)
    else:
        #print(sp)
        p['user'] = sp[3]
        p['ip']   = sp[5]
        p['port'] = sp[7]
        p['attack'] = 'vaild user'
        psw_f.append(p)

In [16]:
len(psw_f)

85372

In [17]:
import pandas as pd

In [18]:
lpd = pd.DataFrame(psw_f)

In [19]:
lpd

,month,day,time,daemon,comment,user,ip,port,attack
0,фев,10,21:12:24,sshd[1160]:,Failed password for root from 221.131.165.86 p...,root,221.131.165.86,49460,root
1,фев,10,21:12:28,sshd[1160]:,Failed password for root from 221.131.165.86 p...,root,221.131.165.86,49460,root
2,фев,10,21:12:32,sshd[1160]:,Failed password for root from 221.131.165.86 p...,root,221.131.165.86,49460,root
3,фев,10,21:12:36,sshd[1183]:,Failed password for root from 54.38.180.93 por...,root,54.38.180.93,40074,root
4,фев,10,21:12:37,sshd[1186]:,Failed password for root from 221.131.165.86 p...,root,221.131.165.86,36018,root
...,...,...,...,...,...,...,...,...,...
85367,фев,19,16:36:55,sshd[15952]:,Failed password for root from 106.54.74.188 po...,root,106.54.74.188,34588,root
85368,фев,19,16:37:14,sshd[16073]:,Failed password for root from 64.225.53.31 por...,root,64.225.53.31,45720,root
85369,фев,19,16:37:32,sshd[16175]:,Failed password for root from 13.66.131.233 po...,root,13.66.131.233,57838,root
85370,фев,19,16:38:49,sshd[16652]:,Failed password for root from 176.109.0.30 por...,root,176.109.0.30,40475,root


In [22]:
lpd.groupby('user').count().sort_values(by='ip', ascending=False)

,month,day,time,daemon,comment,ip,port,attack
user,,,,,,,,
root,39066,39066,39066,39066,39066,39066,39066,39066
admin,3330,3330,3330,3330,3330,3330,3330,3330
user,927,927,927,927,927,927,927,927
test,881,881,881,881,881,881,881,881
ubuntu,602,602,602,602,602,602,602,602
...,...,...,...,...,...,...,...,...
mph,1,1,1,1,1,1,1,1
mpi,1,1,1,1,1,1,1,1
mpiuser,1,1,1,1,1,1,1,1


In [23]:
lpd.to_csv('data/fail_password_attempt.csv')

In [25]:
import sqlite3
con = sqlite3.connect('data/fail_password_attempt.sqlite')
lpd.to_sql('table', con)

85372

In [26]:
con.close()